# DOF gradients

Compute the final gradients needed for the jacobian calculation.

In [2]:
import numpy as np
import sympy
import tensor_tools as tt

In [4]:
zero = sympy.S(0)
one  = sympy.S(1)

delta_Kalpha = tt.form_tensor('_',[3,12])

for i in range(3):
    for j in range(12):
        delta_Kalpha[i][j] = zero
        
delta_Kalpha[0][0] = one
delta_Kalpha[1][1] = one
delta_Kalpha[2][2] = one

delta_Kalpha

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [5]:
delta_kKalpha = tt.form_tensor('_',[9,12])

for i in range(9):
    for j in range(12):
        delta_kKalpha[i][j] = zero

for i in range(9):
    delta_kKalpha[i][i+3] = one
    
delta_kKalpha

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]

In [6]:
sot_voigt_map = tt.get_voigt_mapping(2)
tot_voigt_map = tt.get_voigt_mapping(3)

In [7]:
N = sympy.symbols('N')

In [8]:
dNdx = [sympy.symbols('dNdx'+str(v+1)) for v in range(3)]

Compute voigt matrix form of $\frac{\partial u_{K,k}}{\partial \hat{u}_{\alpha}}$

In [18]:
dgrad_udU = tt.form_tensor('_',[9,12])

string = ''
for K in range(3):
    for k in range(3):
        Khat = sot_voigt_map.index([K,k])
        
        for alpha in range(12):
            
            dgrad_udU[Khat][alpha] = delta_Kalpha[K][alpha]*dNdx[k]
            
            if(dgrad_udU[Khat][alpha] != zero):
                string += " "*8 + "tripletList.push_back(T({0},{1},dNdx[{2}]));\n".format(Khat,alpha,k)
print(string)
string = ''
for K in range(3):
    for k in range(3):
        Khat = sot_voigt_map.index([K,k])
        
        for alpha in range(12):
            
            dgrad_udU[Khat][alpha] = delta_Kalpha[K][alpha]*dNdx[k]
            
            if(dgrad_udU[Khat][alpha] != zero):
                string += " "*8 + "dgrad_udU({0},{1}) = detadx[{2}];\n".format(Khat,alpha,k)
print(string)
dgrad_udU

        tripletList.push_back(T(0,0,dNdx[0]));
        tripletList.push_back(T(5,0,dNdx[1]));
        tripletList.push_back(T(4,0,dNdx[2]));
        tripletList.push_back(T(8,1,dNdx[0]));
        tripletList.push_back(T(1,1,dNdx[1]));
        tripletList.push_back(T(3,1,dNdx[2]));
        tripletList.push_back(T(7,2,dNdx[0]));
        tripletList.push_back(T(6,2,dNdx[1]));
        tripletList.push_back(T(2,2,dNdx[2]));

        dgrad_udU(0,0) = detadx[0];
        dgrad_udU(5,0) = detadx[1];
        dgrad_udU(4,0) = detadx[2];
        dgrad_udU(8,1) = detadx[0];
        dgrad_udU(1,1) = detadx[1];
        dgrad_udU(3,1) = detadx[2];
        dgrad_udU(7,2) = detadx[0];
        dgrad_udU(6,2) = detadx[1];
        dgrad_udU(2,2) = detadx[2];



[[dNdx1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, dNdx2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, dNdx3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, dNdx3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [dNdx3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [dNdx2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, dNdx2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, dNdx1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, dNdx1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Compute voigt matrix form of $\frac{\partial \phi_{kK} \phi}{\partial \hat{u}_{\alpha}}$

In [17]:
dphidU = tt.form_tensor('_',[9,12])

string = ''
for K in range(3):
    for k in range(3):
        Khat = sot_voigt_map.index([K,k])
        
        for alpha in range(12):
            
            dphidU[Khat][alpha] = delta_kKalpha[Khat][alpha]*N
            
            if(dphidU[Khat][alpha] != zero):
                string += " "*8 + "tripletList.push_back(T({0},{1},N));\n".format(Khat,alpha)
print(string)
string = ''
for K in range(3):
    for k in range(3):
        Khat = sot_voigt_map.index([K,k])
        
        for alpha in range(12):
            
            dphidU[Khat][alpha] = delta_kKalpha[Khat][alpha]*N
            
            if(dphidU[Khat][alpha] != zero):
                string += " "*8 + "dphidU({0:2},{1:2}) = eta;\n".format(Khat,alpha)
print(string)
dphidU

        tripletList.push_back(T(0,3,N));
        tripletList.push_back(T(5,8,N));
        tripletList.push_back(T(4,7,N));
        tripletList.push_back(T(8,11,N));
        tripletList.push_back(T(1,4,N));
        tripletList.push_back(T(3,6,N));
        tripletList.push_back(T(7,10,N));
        tripletList.push_back(T(6,9,N));
        tripletList.push_back(T(2,5,N));

        dphidU( 0, 3) = eta;
        dphidU( 5, 8) = eta;
        dphidU( 4, 7) = eta;
        dphidU( 8,11) = eta;
        dphidU( 1, 4) = eta;
        dphidU( 3, 6) = eta;
        dphidU( 7,10) = eta;
        dphidU( 6, 9) = eta;
        dphidU( 2, 5) = eta;



[[0, 0, 0, N, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, N, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, N, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, N, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, N, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, N, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, N, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, N, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, N]]

Compute voigt matrix form of $\frac{\partial \phi_{kK} \phi_{,l}}{\partial \hat{u}_{\alpha}}$

In [16]:
dgrad_phidU = tt.form_tensor('_',[27,12])

string = ''
n=0
for K in range(3):
    for k in range(3):
        Khat = sot_voigt_map.index([k,K])
        for l in range(3):
            Khatl = tot_voigt_map.index([k,K,l])
            for alpha in range(12):
            
                dgrad_phidU[Khatl][alpha] = delta_kKalpha[Khat][alpha]*dNdx[l]
            
                if(dgrad_phidU[Khatl][alpha] != zero):
                    string += " "*8 + "tripletList.push_back(T({0},{1},dNdx[{2}]));\n".format(Khatl,alpha,l)
                    n += 1
print(n)
print(string)

string = ''
n=0
for K in range(3):
    for k in range(3):
        Khat = sot_voigt_map.index([k,K])
        for l in range(3):
            Khatl = tot_voigt_map.index([k,K,l])
            for alpha in range(12):
            
                dgrad_phidU[Khatl][alpha] = delta_kKalpha[Khat][alpha]*dNdx[l]
            
                if(dgrad_phidU[Khatl][alpha] != zero):
                    string += " "*8 + "dgrad_phidU({0:2},{1:2}) = detadx[{2}];\n".format(Khatl,alpha,l)
                    n += 1
print(n)
print(string)
dgrad_phidU

27
        tripletList.push_back(T(0,3,dNdx[0]));
        tripletList.push_back(T(5,3,dNdx[1]));
        tripletList.push_back(T(4,3,dNdx[2]));
        tripletList.push_back(T(9,11,dNdx[0]));
        tripletList.push_back(T(14,11,dNdx[1]));
        tripletList.push_back(T(13,11,dNdx[2]));
        tripletList.push_back(T(18,10,dNdx[0]));
        tripletList.push_back(T(23,10,dNdx[1]));
        tripletList.push_back(T(22,10,dNdx[2]));
        tripletList.push_back(T(8,8,dNdx[0]));
        tripletList.push_back(T(1,8,dNdx[1]));
        tripletList.push_back(T(3,8,dNdx[2]));
        tripletList.push_back(T(17,4,dNdx[0]));
        tripletList.push_back(T(10,4,dNdx[1]));
        tripletList.push_back(T(12,4,dNdx[2]));
        tripletList.push_back(T(26,9,dNdx[0]));
        tripletList.push_back(T(19,9,dNdx[1]));
        tripletList.push_back(T(21,9,dNdx[2]));
        tripletList.push_back(T(7,7,dNdx[0]));
        tripletList.push_back(T(6,7,dNdx[1]));
        tripletList.push_back(T(2,7,dNdx

[[0, 0, 0, dNdx1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, dNdx2, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, dNdx3, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, dNdx3, 0, 0, 0],
 [0, 0, 0, dNdx3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, dNdx2, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, dNdx2, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, dNdx1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, dNdx1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dNdx1],
 [0, 0, 0, 0, dNdx2, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, dNdx3, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, dNdx3, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dNdx3],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dNdx2],
 [0, 0, 0, 0, 0, 0, dNdx2, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, dNdx1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, dNdx1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dNdx1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, dNdx2, 0, 0],
 [0, 0, 0, 0, 0, dNdx3, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, dNdx3, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dNdx3, 0],
 [0, 0, 0, 